### Creating CATALOG, SCHEMA AND TABLES

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS ctl_crypto;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ctl_crypto.staging_schema;

In [0]:
silver_path="abfss://silver@cryptostoragesy01.dfs.core.windows.net/silver_crypto"
gold_path="abfss://gold@cryptostoragesy01.dfs.core.windows.net/"

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.staging_schema.raw_silver
USING DELTA
LOCATION 'abfss://silver@cryptostoragesy01.dfs.core.windows.net/silver_crypto'

In [0]:
%sql
SELECT * FROM ctl_crypto.staging_schema.raw_silver
LIMIT 10;

In [0]:
%sql
SELECT COUNT(*) FROM ctl_crypto.staging_schema.raw_silver;

## NORMALIZING THE GENERAL TABLE
-- By extracting the fact and dimension tables

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS ctl_crypto.gold_schema;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.gold_schema.dim_roi(
  roi_id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  id STRING,
  roi_time DOUBLE,
  roi_currency STRING,
  roi_percentage DOUBLE,
  last_updated_ts TIMESTAMP
)
USING DELTA
LOCATION 'abfss://gold@cryptostoragesy01.dfs.core.windows.net/dim_roi'

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.gold_schema.dim_crypto(
dim_crypto_id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
id STRING,
name STRING,
symbol STRING,
image STRING,
market_cap_billions DOUBLE,
volume_to_market_cap_ratio DOUBLE,
last_updated_ts TIMESTAMP
)
USING DELTA
LOCATION 'abfss://gold@cryptostoragesy01.dfs.core.windows.net/dim_crypto'

In [0]:
%sql
DROP TABLE IF EXISTS ctl_crypto.gold_schema.fact_crypto

In [0]:
%sql
CREATE TABLE IF NOT EXISTS ctl_crypto.gold_schema.fact_crypto(
id STRING PRIMARY KEY,
market_cap_rank BIGINT,
max_supply DOUBLE,
total_supply DOUBLE,
total_volume DOUBLE,
circulating_supply DOUBLE,
market_cap DOUBLE,
market_cap_change_24h DOUBLE,
market_cap_change_percentage_24h DOUBLE,
price_change_24h DOUBLE,
price_change_percentage_24h DOUBLE,
high_24h DOUBLE,
low_24h DOUBLE,
current_price DOUBLE,
fully_diluted_valuation DOUBLE,
date_partition DATE,
last_updated_ts TIMESTAMP
)
USING DELTA
LOCATION 'abfss://gold@cryptostoragesy01.dfs.core.windows.net/fact_crypto'

In [0]:
%sql
MERGE INTO ctl_crypto.gold_schema.dim_roi AS target
USING (
  SELECT id,
  roi_time,
  roi_currency,
  roi_percentage,
  last_updated_ts,
  row_number() OVER(PARTITION BY id ORDER BY last_updated_ts DESC) AS rn
    FROM (
      SELECT *, row_number() OVER(PARTITION BY id ORDER BY last_updated_ts DESC) AS rn
      FROM ctl_crypto.staging_schema.raw_silver) WHERE rn = 1) AS updated_rn

  ON target.id = updated_rn.id
WHEN MATCHED AND updated_rn.rn = 1 THEN
UPDATE SET
  target.roi_time = updated_rn.roi_time,
  target.roi_currency = updated_rn.roi_currency,
  target.roi_percentage = updated_rn.roi_percentage,
  target.last_updated_ts = updated_rn.last_updated_ts
WHEN NOT MATCHED AND updated_rn.rn = 1 THEN
INSERT (
  id, 
  roi_time, 
  roi_currency, 
  roi_percentage, 
  last_updated_ts
) 
VALUES (
  updated_rn.id, 
  updated_rn.roi_time, 
  updated_rn.roi_currency, 
  updated_rn.roi_percentage, 
  updated_rn.last_updated_ts
);

In [0]:
%sql
MERGE INTO ctl_crypto.gold_schema.dim_crypto AS target
USING (
  SELECT id,
    name,
    symbol,
    image,
    market_cap_billions,
    volume_to_market_cap_ratio,
    last_updated_ts,
    row_number() OVER(PARTITION BY id ORDER BY last_updated_ts DESC) AS rn
    FROM (
      SELECT *, row_number() OVER(PARTITION BY id ORDER BY last_updated_ts DESC) AS rn
      FROM ctl_crypto.staging_schema.raw_silver) WHERE rn = 1) AS updated_rn

  ON target.id = updated_rn.id
WHEN MATCHED AND updated_rn.rn = 1 THEN
UPDATE SET
  target.name= updated_rn.name,
  target.symbol = updated_rn.symbol,
  target.image = updated_rn.image,
  target.market_cap_billions = updated_rn.market_cap_billions,
  target.volume_to_market_cap_ratio = updated_rn.volume_to_market_cap_ratio,
  target.last_updated_ts = updated_rn.last_updated_ts
WHEN NOT MATCHED AND updated_rn.rn = 1 THEN
INSERT (
  id, 
  name, 
  symbol, 
  image, 
  market_cap_billions, 
  volume_to_market_cap_ratio, 
  last_updated_ts
)
VALUES (
  updated_rn.id, 
  updated_rn.name, 
  updated_rn.symbol, 
  updated_rn.image, 
  updated_rn.market_cap_billions, 
  updated_rn.volume_to_market_cap_ratio, 
  updated_rn.last_updated_ts
);

In [0]:
%sql
MERGE INTO ctl_crypto.gold_schema.fact_crypto AS target
USING (
  SELECT 
  id,
  market_cap_rank,
  max_supply,
  total_supply,
  total_volume,
  circulating_supply,
  market_cap,
  market_cap_change_24h,
  market_cap_change_percentage_24h,
  price_change_24h,
  price_change_percentage_24h,
  high_24h,
  low_24h,
  current_price,
  fully_diluted_valuation,
  date_partition,
  last_updated_ts,
  row_number() OVER(PARTITION BY id ORDER BY last_updated_ts DESC) AS rn
  FROM (
    SELECT *, row_number() OVER(PARTITION BY id ORDER BY last_updated_ts DESC) AS rn
    FROM ctl_crypto.staging_schema.raw_silver) WHERE rn = 1) AS updated_rn

ON target.id = updated_rn.id
WHEN MATCHED AND updated_rn.rn = 1 THEN
UPDATE SET
  target.market_cap_rank = updated_rn.market_cap_rank,
  target.max_supply = updated_rn.max_supply,
  target.total_supply = updated_rn.total_supply,
  target.total_volume = updated_rn.total_volume,
  target.circulating_supply = updated_rn.circulating_supply,
  target.market_cap = updated_rn.market_cap,
  target.market_cap_change_24h = updated_rn.market_cap_change_24h,
  target.market_cap_change_percentage_24h = updated_rn.market_cap_change_percentage_24h,
  target.price_change_24h = updated_rn.price_change_24h,
  target.price_change_percentage_24h = updated_rn.price_change_percentage_24h,
  target.high_24h = updated_rn.high_24h,
  target.low_24h = updated_rn.low_24h,
  target.current_price = updated_rn.current_price,
  target.fully_diluted_valuation = updated_rn.fully_diluted_valuation,
  target.date_partition = updated_rn.date_partition,
  target.last_updated_ts = updated_rn.last_updated_ts
WHEN NOT MATCHED AND updated_rn.rn = 1 THEN
INSERT (
  id, 
  market_cap_rank, 
  max_supply, 
  total_supply, 
  total_volume, 
  circulating_supply, 
  market_cap, 
  market_cap_change_24h, 
  market_cap_change_percentage_24h, 
  price_change_24h, 
  price_change_percentage_24h, 
  high_24h, 
  low_24h, 
  current_price, 
  fully_diluted_valuation, 
  date_partition, 
  last_updated_ts
) 
VALUES (
  updated_rn.id, 
  updated_rn.market_cap_rank, 
  updated_rn.max_supply, 
  updated_rn.total_supply, 
  updated_rn.total_volume, 
  updated_rn.circulating_supply, 
  updated_rn.market_cap, 
  updated_rn.market_cap_change_24h, 
  updated_rn.market_cap_change_percentage_24h, 
  updated_rn.price_change_24h, 
  updated_rn.price_change_percentage_24h, 
  updated_rn.high_24h, 
  updated_rn.low_24h, 
  updated_rn.current_price, 
  updated_rn.fully_diluted_valuation, 
  updated_rn.date_partition, 
  updated_rn.last_updated_ts
);